In [4]:
import cv2
import numpy as np
import scipy.signal as sp

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [5]:
#
# Parameters
#
SQUARE_SIZE = 5
HEIGHT_SCALE = 16
HEIGHT_WIDTH = 1
BLACK_OFFSET = HEIGHT_WIDTH*HEIGHT_SCALE + SQUARE_SIZE

#
# Colors
#
clr_base_tile = 70
clr_tile_border = 25
clr_south_wall = 40
clr_west_wall = 50

#
# Objects
#
obj_base_tile = np.zeros((SQUARE_SIZE, SQUARE_SIZE))
obj_base_tile.fill(clr_base_tile)
obj_base_tile[0, :] = clr_tile_border
obj_base_tile[:, SQUARE_SIZE - 1] = clr_tile_border

In [49]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    
    # convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # initialize the face recognizer (default face haar cascade)
    # download the XML file from here: https://github.com/opencv/opencv/tree/master/data/haarcascades
    face_cascade = cv2.CascadeClassifier("cascades/haarcascade_frontalface_alt.xml")
    
    # detect all the faces in the image
    faces = face_cascade.detectMultiScale(gray)
    
    # for every face, draw a blue rectangle
    for x, y, width, height in faces:
        #cv2.rectangle(gray, (x, y), (x + width, y + height), color=(255, 0, 0), thickness=2)
        
        # Get the face and show it
        face = gray[y-BLACK_OFFSET:y+height+BLACK_OFFSET, x-BLACK_OFFSET:x+width+BLACK_OFFSET]
        cv2.imshow("face", face)
    
        # perform the canny edge detector to detect image edges and show them
        #edges = cv2.Canny(face, threshold1=40, threshold2=75)
        #cv2.imshow("edges", edges)
            
        #
        # Convert input black and white image to small picture and mean convolve
        #
        filter = np.zeros((SQUARE_SIZE, SQUARE_SIZE))
        filter.fill(1.0 / SQUARE_SIZE**2)

        small_img = sp.convolve2d(face, filter[::-1, ::-1], mode="valid")[::SQUARE_SIZE, ::SQUARE_SIZE]
        
        #
        # Scale the image of tiles
        #
        scaled_small_img = small_img - small_img.min()
        scale = np.linspace(0, scaled_small_img.max(), HEIGHT_SCALE + 1)
        scaled_small_img = np.digitize(scaled_small_img, scale) - 1
        
        #
        # Main body of the effect
        #

        # This one makes the black border!
        final_img = np.zeros((face.shape[0] + BLACK_OFFSET, face.shape[1] + BLACK_OFFSET)).astype(np.uint8)

        # Actually fills the image with tiles and height
        for ii in range(scaled_small_img.shape[0]):
            for jj in range(scaled_small_img.shape[1] - 1, -1, -1):
                height_value = int(scaled_small_img[ii, jj])

                if (height_value > 0):
                    left_lower_row = (ii + 1)*SQUARE_SIZE - 1 + BLACK_OFFSET
                    left_lower_col = jj*SQUARE_SIZE

                    for kk in range(0, height_value*HEIGHT_WIDTH):
                        final_img[left_lower_row - kk, (left_lower_col + kk):(left_lower_col + SQUARE_SIZE + kk)] = clr_south_wall
                        final_img[(left_lower_row - SQUARE_SIZE - kk + BLACK_OFFSET):(left_lower_row - kk - 1 + BLACK_OFFSET), left_lower_col + kk] = clr_west_wall

                    final_img[(left_lower_row - SQUARE_SIZE - kk):(left_lower_row - kk), 
                              (left_lower_col + kk):(left_lower_col + SQUARE_SIZE + kk)] = obj_base_tile

        # Show the result
        cv2.imshow("Effect", final_img)
    
    # show the image
    cv2.imshow("image", gray)
    
    if (cv2.waitKey(1) & 0xFF == ord("q")):
        break

cap.release()
cv2.destroyAllWindows()

In [46]:
cap.release()
cv2.destroyAllWindows()